<br>
<br>
#  <strong>알고리즘 트레이딩 시스템의 구현</strong> 

<strong>알고리즘 트레이딩 시스템 블록</strong>
1. <strong>알파모델</strong> : 주가 예측
2. <strong>리스크모델(Risk Model)</strong> : 알파모델 예측이 틀릴경우 손실을 계산
3. <strong>거래비용모델(Transaction Cost Model)</strong> : 거래비용을 계산
4. <strong>포트폴리오모델(Portfolio Model)</strong> : 알파/리스크/거래비용 모델을 종합하여 거래를 최종판단
5. <strong>실행모델(Executaion Model)</strong> : 포트폴리오 결정에 따라 실제로 거래를 실행

<img src="https://doi.ieeecomputersociety.org/cms/Computer.org/dl/mags/co/2011/11/figures/mco2011110061a.gif" width="400" align="left">

<br>
## <strong>1 구현 시스템 개요
1. 평균회귀, 머신러닝 등을 활용하여 알파 모델을 생성생성한다
2. 알파모델을 테스트 튜닝 뒤 실전에 적용하면 ,시간에 따라 점차 수익성이 낮아진다
3. 이러한 Life Cycle을 '시간가치 감소효과'라고 한다
    1. 컴퓨터 트레이딩의 범람으로, 다른 모델에 의해 나의 모델이 희석된다
    2. 주가가 '표류하는 랜덤워크'를 따르므로, 시가니 지날수록 변동성이 커진다

Sytem Block
1. 데이터크롤러 (Data Crawler)
1. 데이터베이스 (MySQL, PostgreSQL)
1. 평균회귀모델/ 머신러닝모델
1. alpha 모델
1. Portfolio Builder
1. Trader 
1. Back Tester : 과거데이터를 적용하여 예측 적중률을 계산

<br>
## <strong>2 Data Crawler
<strong>데이터 크롤러</strong> : Yahoo Finance/ Google Finance -> PostgreSQL 에 저장

In [22]:
from pandas_datareader import get_data_google, get_data_yahoo
# get_data_yahoo('005930.ks','2017-01-01','2017-11-01')

<br>
## <strong>3 alpha 모델의 구현
1. 모델의 적합도 판단 
2. 포지션 결정 : 매수(Long Position), 매도(Short Position)을 결정한다

### <strong>01 평균회귀 모델의 구현
모델의 랜덤워크, 회귀모델 판단

In [23]:
#class AlphaModel():
# def __init__(self):
#     self.dbhandler = services.get('dbhandler')
#     self.dbreader = services.get('dbreader')
#     self.predictor = services.get('predictor')
#     self.config = services.get('configurator')

In [24]:
# 평균회귀 적합도 판단모델
#class MeanReversionModel(AlphaModel):
def calcADF(self, df):
    adf_result = ts.adfuller(df)
    ciritical_values = adf_result[4]
    #print ciritical_values
    return adf_result[0], ciritical_values['1%'],ciritical_values['5%'], ciritical_values['10%']

def calcHurstExponent(self,df,lags_count=100):
    lags = range(2, lags_count)
    ts = np.log(df)
    tau = [np.sqrt(np.std(np.subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    poly = np.polyfit(np.log(lags), np.log(tau), 1)
    result = poly[0]*2.0
    return result

def calcHalfLife(self,df):
    price = pd.Series(df)  
    lagged_price = price.shift(1).fillna(method="bfill")  
    delta = price - lagged_price  
    beta = np.polyfit(lagged_price, delta, 1)[0] 
    half_life = (-1*np.log(2)/beta) 
    return half_life

# 이동평균과 이동평균 표준편차값을 이용하여, 매도/매수 포지션을 결정한다
# self.threshold : 이동평균과 현재주가의 차이가, 이동평균의 표준편차 보아 크고/ 작음을 판단
def determinePosition(self,df,column,row_index,verbose=False):
    current_price = df.loc[row_index,column]
    df_moving_average = pd.rolling_mean(df.loc[0:row_index,column],window=self.window_size)
    df_moving_average_std = pd.rolling_std(df.loc[0:row_index,column],window=self.window_size)
    #print (df.loc[0:row_index,column], moving_average, df_moving_average_std)
    moving_average = df_moving_average[row_index]
    moving_average_std = df_moving_average_std[row_index]
    price_arbitrage = current_price - moving_average
#     if verbose: 
#         print("diff={}, price={}, moving_average={}, moving_average_std={}".format(
#             price_arbitrage, current_price, moving_average, moving_average_std)
#     if abs(price_arbitrage) > moving_average_std * self.threshold:
#         if np.sign(price_arbitrage)>0:
#             return SHORT
#         else:
#             return LONG
#     return HOLD

### <strong>02 머신러닝 모델의 구현
앞장의 머신러닝 모델을 통해 구현한다

In [25]:
#class MachineLearningModel(AlphaModel):
def calcScore(self,split_ratio=0.75,time_lags=10):
    return self.predictor.trainAll(split_ratio=split_ratio,time_lags=time_lags )

# Voting 기법을 사용하여 매수/매도를 판단 : 3가지 예측모델의 판단 결과를 투표로 최종결정한다
def determinePosition(self,code,df,column,row_index,verbose=False):
    if (row_index-1) < 0:
        return HOLD
    current_price = df.loc[row_index-1,column]
    prediction_result = 0
    for a_predictor in ['logistic','rf','svm']:
        predictor = self.predictor.get(code,a_predictor)
        pred,pred_prob = predictor.predict([current_price])
        #print("predictor=%s, price=%s, pred=%s, pred_proba=%s" % (a_predictor,current_price,pred[0],pred_prob[0]))
        prediction_result += pred[0]
        #print (prediction_result[a_predictor])
    #print("price=%s, pred_result=%s" % (current_price,prediction_result))
    if prediction_result>1:
        return LONG
    else:
        return SHORT

<br>
## <strong>4 포트폴리오 빌더 구현
모델구현과 함께, 이에 적합한 종목을 골라내는 것 또한 중요한 과제이다
1. 종목별 모델의 적합성 평가
1. 적합성 평가 결과 순위를 지정
1. 정해진 순위로 상위 % 종목을 추출

### <strong>01 평균회귀 모델의 종목선정
ADF, 허스트지수, Half-Life 계수를 통해서 판단이 가능하다

종목별 위의 3가지 판단을 계산하고, 그 결과를 DataFrame으로 출력한다

In [26]:
#class PortfolioBuilder():
# def __init__(self):
#     self.dbhandler = services.get('dbhandler')
#     self.dbreader = services.get('dbreader')
#     self.predictor = services.get('predictor')
#     self.config = services.get('configurator')
#     self.mean_reversion_model = services.get('mean_reversion_model')
#     self.machine_learning_model = services.get('machine_learning_model')

def loadDataFrame(self,code): # SQL에서 데이터 추출
    sql = "select * from prices"
    sql += " where code='%s'" % (code)
    sql += " and price_date between '%s' and '%s' " % (self.config.get('start_date'),self.config.get('end_date'))
    df = pd.read_sql(sql,self.dbhandler.conn)
    return df

In [27]:
def assessADF(self,test_stat,adf_1,adf_5,adf_10):
    if test_stat<adf_10:  return 3
    elif test_stat<adf_5: return 2
    elif test_stat<adf_1: return 1
    return 0

def assessHurst(self,hurst):
    if hurst>0.4:   return 0
    if hurst<0.1:   return 3
    elif hurst<0.2: return 2
    return 1

def assessHalflife(self,percentile,halflife):
    for index in range(len(percentile)):
        if halflife<=percentile[index]:
            if index<2:   return 3
            elif index<3: return 2
            elif index<4: return 1
    return 0

In [28]:
def doStationarityTest(self,column,lags_count=100):
    rows_code = self.dbreader.loadCodes(limit = self.config.get('data_limit'))
    test_result = {'code':[], 'company':[], 'adf_statistic':[], 
                   'adf_1':[],'adf_5':[],'adf_10':[], 'hurst':[],'halflife':[]}
    index = 1
    for a_row_code in rows_code:
        code = a_row_code[0]
        company = a_row_code[1]
        print("... %s of %s : Testing Stationarity on %s %s" % (index,len(rows_code),code,company))
        a_df = self.loadDataFrame(code)
        a_df_column = a_df[column]
        if a_df_column.shape[0]>0:
            test_result['code'].append(code)
            test_result['company'].append(company)
            test_result['hurst'].append(self.mean_reversion_model.calcHurstExponent(a_df_column,lags_count))
            test_result['halflife'].append(self.mean_reversion_model.calcHalfLife(a_df_column))
            test_stat, adf_1,adf_5,adf_10 = self.mean_reversion_model.calcADF(a_df_column)
            test_result['adf_statistic'].append(test_stat)
            test_result['adf_1'].append(adf_1)
            test_result['adf_5'].append(adf_5)
            test_result['adf_10'].append(adf_10)
        index += 1
    df_result = pd.DataFrame(test_result)
    return df_result

In [29]:
# 계산된 결과에 따른 종목별 적합도 순위 결정
def rankStationarity(self,df_stationarity):
    df_stationarity['rank_adf'] = 0
    df_stationarity['rank_hurst'] = 0
    df_stationarity['rank_halflife'] = 0
    halflife_percentile = np.percentile(
        df_stationarity['halflife'], np.arange(0, 100, 10)) # quartiles
    for row_index in range(df_stationarity.shape[0]):
        df_stationarity.loc[row_index,'rank_adf'] =\
            self.assessADF(df_stationarity.loc[row_index,'adf_statistic'],
                           df_stationarity.loc[row_index,'adf_1'],
                           df_stationarity.loc[row_index,'adf_5'],
                           df_stationarity.loc[row_index,'adf_10'])
        df_stationarity.loc[row_index,'rank_hurst'] =\
            self.assessHurst(df_stationarity.loc[row_index,'hurst'])
        df_stationarity.loc[row_index,'rank_halflife'] =\
            self.assessHalflife(halflife_percentile, 
                                df_stationarity.loc[row_index,'halflife'])
    df_stationarity['rank'] = df_stationarity['rank_adf'] +\
                              df_stationarity['rank_hurst'] +\
                              df_stationarity['rank_halflife']
    return df_stationarity

In [30]:
# 순위결과에 따른 종목선정 : ratio (선정비율)
def buildUniverse(self, df_stationarity, column,  ratio):
    percentile_column = np.percentile(df_stationarity[column], np.arange(0, 100, 10))
    ratio_index = np.trunc(ratio * len(percentile_column))
    universe = {}
    for row_index in range(df_stationarity.shape[0]):		
        percentile_index = getPercentileIndex(
            percentile_column, df_stationarity.loc[row_index,column])
        if percentile_index >= ratio_index:
            universe[df_stationarity.loc[row_index,'code']] =\
                        df_stationarity.loc[row_index,'company']
    return universe

### <strong>02 머신러닝 모델의 종목선정
앞장의 머신러닝 모델을 통해 구현한다

In [31]:
def doMachineLearningTest(self,split_ratio=0.75,lags_count=10):
    return self.machine_learning_model.calcScore(
            split_ratio = split_ratio, time_lags = lags_count )

In [32]:
def assessMachineLearning(self,percentile,halflife):
    for index in range(len(percentile)):
        if halflife<=percentile[index]:
            if index<2:   return 3
            elif index<3: return 2
            elif index<4: return 1
    return 0

### <strong>03 평균회귀 모델의 종목선정
ADF, 허스트지수, Half-Life 계수를 통해서 판단이 가능하다

종목별 위의 3가지 판단을 계산하고, 그 결과를 DataFrame으로 출력한다

In [33]:
def rankMachineLearning(self,df_machine_learning):
    def listed_columns(arr,prefix):
        result = []
        for a_item in arr:
            result.append( prefix % (a_item) )
        return result
    mr_models = ['logistic','rf','svm']
    for a_predictor in mr_models:
        df_machine_learning['rank_%s' % (a_predictor)] = 0
    percentiles = {}
    for a_predictor in mr_models:
        percentiles[a_predictor] = np.percentile(df_machine_learning[a_predictor], np.arange(0, 100, 10))
        for row_index in range(df_machine_learning.shape[0]):
            df_machine_learning.loc[row_index,'rank_%s' % (a_predictor)] =\
                    self.assessMachineLearning(percentiles[a_predictor],
                                               df_machine_learning.loc[row_index,a_predictor])
    df_machine_learning['total_score'] = df_machine_learning[mr_models].sum(axis=1)
    df_machine_learning['rank'] = df_machine_learning[ listed_columns(mr_models,'rank_%s') ].sum(axis=1)
    return df_machine_learning

<br>
## <strong>5 트레이더 구현
PortFolio 클래스에 담긴 종목들을 가상으로 적용해서 매수/ 매도 포지션을 기록한다